In [1]:
import pandas as pd
import geopandas as gdp

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
opt_dir = '../data/raw'
crime = pd.read_excel(f"{opt_dir}/external_data/recorded_offences.xlsx", sheet_name="Table 03", nrows=74721)
crime

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Offence Count
0,2021,December,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,1
1,2021,December,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1
2,2021,December,Alpine,3691,Dederang,B Property and deception offences,B50 Deception,B53 Obtain benefit by deception,1
3,2021,December,Alpine,3691,Kergunyah South,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,1
4,2021,December,Alpine,3691,Kergunyah South,E Justice procedures offences,E10 Justice procedures,E14 Pervert the course of justice or commit pe...,2
...,...,...,...,...,...,...,...,...,...
74716,2020,December,Yarriambiack,3491,Patchewollock,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1
74717,2020,December,Yarriambiack,3491,Patchewollock,B Property and deception offences,B20 Property damage,B21 Criminal damage,3
74718,2020,December,Yarriambiack,3491,Patchewollock,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,2
74719,2020,December,Yarriambiack,3491,Patchewollock,B Property and deception offences,B40 Theft,B42 Steal from a motor vehicle,1


In [106]:
crime = crime[['Postcode', 'Offence Count']]
crime_agg_yr_pc = crime.groupby(['Postcode'], as_index=False).sum()
crime_agg_yr_pc = crime_agg_yr_pc.rename(columns={'Postcode' : 'postcode'})
crime_agg_yr_pc

,postcode,Offence Count
0,3000,37272
1,3002,2429
2,3003,2708
3,3004,2415
4,3005,174
...,...,...
691,3990,5
692,3991,118
693,3992,206
694,3995,2179


In [107]:
population = pd.read_csv('../data/raw/external_data/population_complete.csv', index_col=0)
population = population.drop(columns=['ERP 2020', 'Area km2', 'density 2021 persons/km2'])

In [117]:
postcode = pd.read_csv('../data/raw/external_data/australian_postcodes.csv')
postcode = postcode[postcode['state'] == 'VIC']
postcode_sa2 = postcode[['postcode', 'SA2_MAINCODE_2016']]
postcode_sa2 = postcode_sa2.drop_duplicates()
postcode_sa2 = postcode_sa2.rename(columns={'SA2_MAINCODE_2016' : 'SA2 code'})

In [135]:
postcode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3534 entries, 6151 to 18441
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3534 non-null   int64  
 1   postcode           3534 non-null   int64  
 2   locality           3534 non-null   object 
 3   state              3534 non-null   object 
 4   long               3534 non-null   float64
 5   lat                3534 non-null   float64
 6   dc                 3481 non-null   object 
 7   type               3320 non-null   object 
 8   status             3527 non-null   object 
 9   sa3                3515 non-null   float64
 10  sa3name            3515 non-null   object 
 11  sa4                3515 non-null   float64
 12  sa4name            3515 non-null   object 
 13  region             3531 non-null   object 
 14  Lat_precise        3534 non-null   float64
 15  Long_precise       3534 non-null   float64
 16  SA1_MAINCODE_2011  3

In [136]:
len(postcode['SA2_MAINCODE_2016'].unique())

444

In [125]:
postcode_population = pd.merge(postcode_sa2, population, on='SA2 code', how='left')
postcode_population_agg = postcode_population.groupby('postcode', as_index=False).sum()
postcode_population_agg

,postcode,SA2 code,ERP 2021
0,3000,206041122.0,47192.0
1,3001,206041122.0,47192.0
2,3002,206041119.0,5803.0
3,3003,206041127.0,3.0
4,3004,412082251.0,32825.0
...,...,...,...
742,8111,206041122.0,47192.0
743,8120,206041122.0,47192.0
744,8205,206041122.0,47192.0
745,8785,212041311.0,32936.0


In [126]:
crime_sa2 = pd.merge(crime_agg_yr_pc, postcode_population_agg, on='postcode', how='left')

In [132]:
crime_sa2['scaled_crime_rate'] = crime_sa2['Offence Count'] / crime_sa2['ERP 2021']

In [133]:
crime_sa2.to_csv("../data/raw/external_data/crime_completed.csv")